In [25]:
# dependencies
using Polynomials

# First Implementation 
Direction: 
- Evaluate the function at various points around the edges of the rectangle. 
- Count how many cycles the argument completes while traversing around the rectangle.

Spot rectangle, and how many zeros are inside.
By tracking the winding number, we can locate the number of zeros in the holomorphic function

Wind(arg) = 1/(2*pi*im) * integrate(1/(x+im*y))

In [67]:
# define data structure of rectangle
struct Rect 
    btmlft::Any
    uprght::Any
end

struct Step
    height::Float32
    width::Float32
    step::Float32
end

In [68]:
# calculate height and width
function parseInput(input) 
    height = input.uprght[2] - input.btmlft[2]
    width = input.uprght[1] - input.btmlft[1]
    n = 2000 # change n accordingly so argument analysis is comprehensive
    step = height/n
    info = Step(height, width, step)
end

parseInput (generic function with 1 method)

In [69]:
# evaluate the argument at various points starting at bottomleft and traversing one round
# store arguments in array

function argBox(f, input, info)
    # bottom left to top left
    i = input.btmlft[2]
    argArray = []
    while i < input.uprght[2] 
        arg = angle(f(input.btmlft[1]+im*i))  # this is where the input function comes in for evaluation. Check multipoint.ipynb
        complexpoint = input.btmlft[1]+im*i
        push!(argArray,arg)
        i += info.step
    end
    
    # top left to top right
    j = input.btmlft[1]
    while j < input.uprght[1]
        arg = angle(f(j+im*input.uprght[2]))
        complexpoint = j+im*input.uprght[2]
        push!(argArray,arg)
        j += info.step
    end
    
    # top right to bottom right
    i = input.uprght[2]
    while i > input.btmlft[2]
        arg = angle(f(input.uprght[1]+im*i))
        complexpoint = input.uprght[1]+im*i
        push!(argArray,arg)
        i -= info.step
    end
    
    # bottom right to bottom left
    j = input.uprght[1]
    while j > input.btmlft[1]
        arg = angle(f(j+im*input.btmlft[2]))
        complexpoint = j+im*input.btmlft[2]
        push!(argArray,arg)
        j -= info.step
    end
    
    # add the starting point to make the loop closed
    arg = angle(f(input.btmlft[1]+im*input.btmlft[2]))
    push!(argArray,arg)
    complexpoint = input.btmlft[1]+im*input.btmlft[2]
    return argArray
end


argBox (generic function with 1 method)

In [70]:
# spot jumps (~>= 2pi or 6)
# the output of argBox(input) is an array of the arguments
# use a for loop to check if any of the pairs match the criteria
# increase count and return the final count. This is the number of zeros in the locus


function countJump(f, arr, info)
    len = length(arr)
    count = 0
    add = []
    dec = []
    
    for i in 1:(len - 1)
        if arr[i] - arr[i+1] > 4 # slightly less than 2pi
            count -= 1
            push!(dec, (arr[i],arr[i+1],i))
        elseif arr[i] - arr[i+1] < -4 # decrement if it goes clockwise (less than -6)
            count += 1
            push!(add, (arr[i],arr[i+1],i))
        end
    end
    return count
end

countJump (generic function with 1 method)

In [83]:
# subdivide rectangle into four
# run recursion of countJump for sub-rectangles that have zeros
# repeat recursion until all of the jumps are found. 
# return the bottom left coordinates of the squares
    
# halt when we reach desired accuracy instead of using count 
function divideRec(f, input, info, res, count)
    height = info.height
    width = info.width
    
    # bottom left square
    x_bl = input.btmlft[1]
    y_bl = input.btmlft[2]
    x_tr = width/2 + input.btmlft[1]
    y_tr = height/2 + input.btmlft[2]
    new_input = ((x_bl,y_bl),(x_tr, y_tr))
    new_info = parseInput(new_input)
    new_arr = argBox(f, new_input, new_info)

    if countJump(f, new_arr, new_info) == 0 && count != 0 # add count to know when to push coordinate to result array
        push!(res, (input.btmlft[1],input.btmlft[2]))
        res
    elseif countJump(f, new_arr, new_info) > 0 # change to != 0 after debugging
        divideRec(f, new_input, new_info, (x_bl, y_bl), count+1) # res: the bottom left coordinate?
    end

    # top left square
    x_bl = input.btmlft[1]
    y_bl = input.btmlft[2]+height/2
    x_tr = width/2 + input.btmlft[1]
    y_tr = input.uprght[2]
    new_input = ((x_bl, y_bl),(x_tr, y_tr))
    new_info = parseInput(new_input)
    new_arr = argBox(f, new_input, new_info)
    
    if countJump(f, new_arr, new_info) == 0 && count != 0
        push!(res, (input.btmlft[1],input.btmlft[2]))
        res
    elseif countJump(f, new_arr, new_info) > 0 # change to != 0 after debugging
        divideRec(f, new_input, new_info, (x_bl, y_bl), count+1) 
    end
    
    # top right square
    x_bl = width/2 + input.btmlft[1]
    y_bl = height/2 + input.btmlft[2]
    x_tr = input.uprght[1]
    y_tr = input.uprght[2]
    new_input = ((x_bl, y_bl),(x_tr, y_tr))
    new_info = parseInput(new_input)
    new_arr = argBox(f, new_input, new_info)
    
    if countJump(f, new_arr, new_info) == 0 && count != 0
        push!(res, (input.btmlft[1],input.btmlft[2]))
        res
    elseif countJump(f, new_arr, new_info) > 0 # change to != 0 after debugging
        divideRec(f, new_input, new_info, (x_bl, y_bl)) 
    end
    
    # bottom right square
    x_bl = width/2 + input.btmlft[1]
    y_bl = input.btmlft[2]
    x_tr = input.uprght[1]
    y_tr = height/2 + input.btmlft[2]
    new_input = ((x_bl, y_bl),(x_tr, y_tr))
    new_info = parseInput(new_input)
    new_arr = argBox(f, new_input, new_info)
    
    if countJump(f, new_arr, new_info) == 0 && count != 0
        push!(res, (input.btmlft[1],input.btmlft[2]))
        res
    elseif countJump(f, new_arr, new_info) > 0 # change to != 0 after debugging
        divideRec(f, new_input, new_info, (x_bl, y_bl))
    end

end



divideRec (generic function with 2 methods)

## Unit Tests
Test the algorithm properly. Polynomials are easy to put the zeros exactly where you want them. Try multiplying another function which has no zero in the region to make it more computationally expensive.

In [87]:
# first unit test with 4 zeros
t1 = fromroots([-1+2im,2+4im,-3im, 4+6im])
input = Rect((1,1), (10,10))
info = parseInput(input) # handles steps
arr = argBox(t1, input, info)
countJump(t1, arr, info) == 2 # (1,4), (4,6)

divideRec(t1, input, info, [], 0)

ErrorException: type Tuple has no field uprght

In [64]:
t2 = fromroots([1+2im,1+4im,-1+3im, 4+6im])  # zero lying on borders can't be detected by code if 3im
input2 = Rect((-10,0), (0,10))
info2 = parseInput(input2)
arr2 = argBox(t2, input2, info2)
countJump(t2, arr2, info2) == 1

true